In [1]:
import sqlite3
import pandas as pd
import numpy as np
import io

In [2]:
df = pd.read_csv('data', encoding='utf-16', sep=';', low_memory=False)
# removendo linhas com o cabeçalho (devivo a concatenacao de todos os .csv)
df = df[df['Pac'] != 'Pac']
# adicionando codigo de cada item ao dataframe (chave da entidade Item)
df['CodItem'] = df['ItemMarca'].astype(str) + df['ItemModelo'].astype(str)

In [3]:
def transform_date(s):
    try:
        data = s.split("/")
        return data[2]+"-"+data[1]+"-"+data[0]
    except:
        return s

df['DataConcessao'] = df['DataConcessao'].apply(transform_date)
df['DataValidade'] = df['DataValidade'].apply(transform_date)
df['DataCancelamento'] = df['DataCancelamento'].apply(transform_date)
df['ItemDataAlteracao'] = df['ItemDataAlteracao'].apply(transform_date)

In [5]:
df['CNPJ'] = df['CnpjCpf']
df['UF'] = df['Uf']
df['Portaria'] = df['PortariaInmetro']

In [6]:
conn = sqlite3.connect("RegistroObjetos2.db")
c = conn.cursor()
c.execute('PRAGMA foreign_keys = 0')
conn.commit()

In [7]:
empresa = ['CNPJ', 'RazaoSocial', 'Email', 'Telefone', 'Logradouro', 'Numero', 'Complemento', 'Cep', 'Bairro', 'Municipio', 'UF']
registros = ['NumeroRegistro', 'CNPJ', 'CodItem', 'Status', 'DataConcessao', 'DataValidade', 'DataCancelamento']
item = ['CodItem', 'ItemMarca', 'ItemModelo', 'ItemDescricao', 'ItemStatus', 'ItemDataAlteracao', 'ItemCodigodeBarra', 'Pac']
objeto = ['Pac', 'Portaria', 'Familia', 'Certificado']

In [10]:
c.execute('drop table if exists Empresa')
conn.commit()
df_empresa = df[empresa]
df_empresa = df_empresa.drop_duplicates('CNPJ')
df_empresa.to_sql('Empresa', con=conn, index_label='id')

In [10]:
c.execute('drop table if exists Registros')
conn.commit()
df_registros = df[registros]
df_registros = df_registros.drop_duplicates('NumeroRegistro')
df_registros.to_sql('Registros', con=conn, index=False)

In [11]:
c.execute('drop table if exists Item')
conn.commit()
df_item = df[item]
df_item = df_item.drop_duplicates('CodItem')
df_item.to_sql('Item', con=conn, index=False)

In [12]:
c.execute('drop table if exists Objeto')
conn.commit()
df_objeto = df[objeto]
df_objeto = df_objeto.drop_duplicates('Pac')
df_objeto.to_sql('Objeto', con=conn, index=False)